In [8]:
from pathlib import Path

import numpy as np
import pandas as pd

In [9]:
pd.options.display.float_format = "{:,.0f}".format

In [10]:
data_dir: Path = Path().home() / "data" / "usaspending"

In [11]:
df = pd.read_csv(data_dir / "ag_ira_awards_withheld.csv")

/tmp/ipykernel_73409/448377676.py:1: DtypeWarning: Columns (16,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_dir / "ag_ira_awards_withheld.csv")


In [12]:
df.head()

,assistance_transaction_unique_key,assistance_award_unique_key,award_id_fain,total_obligated_amount,total_outlayed_amount_for_overall_award,action_date_fiscal_year,treasury_accounts_funding_this_award,program_activities_funding_this_award,recipient_uei,recipient_name,...,primary_place_of_performance_zip_4,prime_award_transaction_place_of_performance_cd_original,prime_award_transaction_place_of_performance_cd_current,cfda_number,cfda_title,assistance_type_description,prime_award_base_transaction_description,business_types_description,max_promised_amount,max_withheld_amount
0,12H3_20227041637148_-NONE-_10.234_1,ASST_NON_20227041637148_12H3,20227041637148,"14,860,449","8,508,901",2022,012-2022/2031-0115-000;012-X-0115-000,"0016: ARP: SOCIALLY DISADVANTAGE FARMERS, RANC...",KB5RHNPHRXZ6,FEDERATION OF SOUTHERN COOPERATIVES/LAND ASSIS...,...,30344-3258,GA-05,GA-05,10,AMERICAN RESCUE PLAN TECHNICAL ASSISTANCE INVE...,COOPERATIVE AGREEMENT (B),THE FEDERATION OF SOUTHERN COOPERATIVES/LAND A...,NONPROFIT WITHOUT 501C3 IRS STATUS (OTHER THAN...,"14,860,449","6,351,548"
1,12H3_20237050541116_-NONE-_10.234_1,ASST_NON_20237050541116_12H3,20237050541116,"4,995,629","4,482,724",2023,012-2022/2031-0115-000,0061: IRA SEC 22007 (E) DISCRIMINATION FINANCI...,X41KZSTTC8K4,"INTERTRIBAL AGRICULTURE COUNCIL, INC",...,59102-4711,MT-02,MT-02,10,AMERICAN RESCUE PLAN TECHNICAL ASSISTANCE INVE...,PROJECT GRANT (B),"FOR OVER 35 YEARS, THE INTERTRIBAL AGRICULTURE...",NONPROFIT WITH 501C3 IRS STATUS (OTHER THAN AN...,"4,995,629","512,904"
2,12H3_20227041637276_-NONE-_10.234_1,ASST_NON_20227041637276_12H3,20227041637276,"8,955,772","3,985,693",2022,012-2022/2031-0115-000;012-X-0115-000,"0016: ARP: SOCIALLY DISADVANTAGE FARMERS, RANC...",KMLRLJRK83M5,RURAL COALITION,...,20005-3842,DC-98,DC-98,10,AMERICAN RESCUE PLAN TECHNICAL ASSISTANCE INVE...,COOPERATIVE AGREEMENT (B),AS PART OF A TEAM OF INTERRELATED USDA COOPERA...,NONPROFIT WITHOUT 501C3 IRS STATUS (OTHER THAN...,"8,955,772","4,970,079"
3,12C3_NR225A12XXXXC009_-NONE-_10.069_-NONE-,ASST_NON_NR225A12XXXXC009_12C3,NR225A12XXXXC009,"7,340,254","3,976,526",2022,012-2023/2031-1004-000;012-X-1004-000,0002: ENVIRONMENTAL QUALITY INCENTIVES PROGRAM...,QJ81C6YXHWF8,ASSOCIATION OF ILLINOIS SOIL & WATER CONSERVAT...,...,NaN,IL-90,NaN,10,CONSERVATION RESERVE PROGRAM,COOPERATIVE AGREEMENT (B),THE OBJECTIVE OF THIS AGREEMENT IS TO POOL THE...,NONPROFIT WITH 501C3 IRS STATUS (OTHER THAN AN...,"7,340,254","3,363,728"
4,12H3_20237050440441_-NONE-_10.234_1,ASST_NON_20237050440441_12H3,20237050440441,"5,000,000","3,936,596",2023,012-2022/2031-0115-000,PM08: PENDING MAPPING,LZHUBAX82H75,SOVEREIGN EQUITY FUND,...,58501-3967,ND-00,ND-00,10,AMERICAN RESCUE PLAN TECHNICAL ASSISTANCE INVE...,PROJECT GRANT (B),TRIBAL EXTENSION SUSTAINABILITY INITIATIVE (TE...,NONPROFIT WITH 501C3 IRS STATUS (OTHER THAN AN...,"5,000,000","1,063,404"


In [13]:
df["prime_award_transaction_recipient_cd_current"] = df["prime_award_transaction_recipient_cd_current"].replace("-", "")
df["prime_award_transaction_recipient_cd_current"] = df["prime_award_transaction_recipient_cd_current"].replace("98", "00")
df["prime_award_transaction_recipient_cd_current"] = df["prime_award_transaction_recipient_cd_current"].replace("90", "Multiple")


In [14]:
eqip_tafs = [
    "012-2023/2031-1004",
    "012-2024/2031-1004",
    "012-2025/2031-1004",
    "012-2026/2031-1004",
]

cta_tafs = [
    "012-2022/2031-1000",
]

In [21]:
conditions = [
    df["cfda_title"] == "AGRICULTURAL CONSERVATION EASEMENT PROGRAM",
    df["cfda_title"] == "CONSERVATION RESERVE PROGRAM",
    df["cfda_title"] == "CONSERVATION STEWARDSHIP PROGRAM",
    df["cfda_title"] == "ENVIRONMENTAL QUALITY INCENTIVES PROGRAM",
    df["cfda_title"] == "REGIONAL CONSERVATION PARTNERSHIP PROGRAM",
    df["cfda_title"] == "RURAL ENERGY FOR AMERICA PROGRAM",
    df["cfda_title"] == "FROM LEARNING TO LEADING: CULTIVATING THE NEXT GENERATION OF DIVERSE FOOD AND AGRICULTURE PROFESSIONALS",

    (
        (df["cfda_title"] == "FARM AND RANCH LANDS PROTECTION PROGRAM")
        & (df["treasury_accounts_funding_this_award"].str.contains("|".join(eqip_tafs)))
    ),
    (
        (df["cfda_title"] == "FARM AND RANCH LANDS PROTECTION PROGRAM")
        & (df["treasury_accounts_funding_this_award"].str.contains("|".join(cta_tafs)))
    ),
    (
        (df["cfda_title"] == "SOIL AND WATER CONSERVATION")
        & (df["treasury_accounts_funding_this_award"].str.contains("|".join(eqip_tafs)))
    ),
    (
        (df["cfda_title"] == "SOIL AND WATER CONSERVATION")
        & (df["treasury_accounts_funding_this_award"].str.contains("|".join(cta_tafs)))
    ),
    (
        (df["cfda_title"] == "AGRICULTURAL AND RURAL ECONOMIC RESEARCH, COOPERATIVE AGREEMENTS AND COLLABORATIONS")
        & (df["treasury_accounts_funding_this_award"].str.contains("|".join(cta_tafs)))
    ),
]

values = [
    "Agricultural Conservation Easement Program",
    "Conservation Reserve Program",
    "Conservation Stewardship Program",
    "Environmental Quality Incentives Program",
    "Regional Conservation Partnership Program",
    "Rural Energy for America Program",
    "NIFA NEXTGEN",
    "Environmental Quality Incentives Program",
    "Conservation Technical Assistance",
    "Environmental Quality Incentives Program",
    "Conservation Technical Assistance",
    "Conservation Technical Assistance",
]

df["program_bucket"] = np.select(conditions, values, default=df["cfda_title"])

df["program_bucket"].value_counts()

program_bucket
Environmental Quality Incentives Program                        13370
Conservation Stewardship Program                                 5397
Rural Energy for America Program                                 3640
Conservation Technical Assistance                                 184
Agricultural Conservation Easement Program                         54
NIFA NEXTGEN                                                       32
AMERICAN RESCUE PLAN TECHNICAL ASSISTANCE INVESTMENT PROGRAM       25
Conservation Reserve Program                                       11
COOPERATIVE FORESTRY ASSISTANCE                                     6
Regional Conservation Partnership Program                           6
DISCRIMINATION FINANCIAL ASSISTANCE PROGRAM                         3
COOPERATIVE EXTENSION SERVICE                                       1
AGRICULTURAL STATISTICS REPORTS                                     1
FORESTRY RESEARCH                                                   1
PARTN

In [22]:
df.to_csv(data_dir / "ag_ira_awards_withheld_program_buckets.csv", index=False)